In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath

**Define LaTeX macros/commands** (they're invisible!)

$\renewcommand{\mchirp}{\mathcal{M}}$

# Construct a population of binaries

In [ ]:
NUM = 1e6
MASS_EXTR = [1e6, 1e10]
# TMAX = (20.0 * YR)
# NFREQS = 100
TMAX = (2.0 * YR)
NFREQS = 100

In [ ]:
fobs_gw = np.arange(1, NFREQS+1) / TMAX
fobs_gw_edges = np.concatenate([fobs_gw - fobs_gw[0]/2, [fobs_gw[-1] + fobs_gw[0]/2]])

masses = zmath.random_power(MASS_EXTR, -3, NUM) * MSOL
redz = 0.1
mrat = 0.3

fig, ax = plot.figax()
kale.dist1d((masses/MSOL), carpet=False)
plt.show()

# Semi-Analytic (SA) Calculation

The GWB characteristic strain spectrum can be calculated **semi-analytically** using a volumetric number-density of sources $n(M, q, z) = dN/dV_c$, as [Phinney 2001, Eq. 5] or [Enoki & Nagashima 2007, Eq. 3.6]:

$$ h_c^2 = \frac{4G}{\pi c^2 f} \int dM \, dq \, dz \, \frac{d^3 n(M, q, z)}{dM \, dq \, dz} \, \left( \frac{dE_{GW}(M, q)}{d f_r}\right)_{f_r = f(1+z)}$$

Assuming circular, GW-driven orbits, this can be simplified to [Enoki & Nagashima 2007, Eq.3.11]:

$$ h_c^2 = \frac{4\pi}{3 c^2} (\pi f)^{-4/3} \int dM \, dq \, dz \, \frac{d^3 n(M, q, z)}{dM \, dq \, dz} \, \frac{(G\mathcal{M})^{5/3}}{(1+z)^{1/3}}$$

Construct a Number-Density

In [ ]:
NBINS = 123
mbin_edges = zmath.spacing(masses, 'log', NBINS+1)
mbin_cents = 0.5 * (mbin_edges[:-1] + mbin_edges[1:])
# calculate comoving-volume
vcom = cosmo.comoving_volume(redz).cgs.value

SHAPE = (NBINS,)
ndens = np.zeros(SHAPE)
ndens, *_ = sp.stats.binned_statistic(masses, None, statistic='count', bins=mbin_edges)
ndens /= np.diff(mbin_edges)
ndens /= vcom

fig, ax = plot.figax()
plot.draw_hist_steps(ax, mbin_edges/MSOL, ndens*MSOL*(MPC**3))
plt.show()

Calculate GWB assuming circular, GW-drive evolution

In [ ]:
mchirp_edges = utils.chirp_mass_mtmr(mbin_edges, mrat)
mchirp_cents = 0.5 * (mchirp_edges[:-1] + mchirp_edges[1:])
integrand = ndens * np.power(NWTG * mchirp_cents, 5.0/3.0) * np.power(1+redz, -1.0/3.0)

gwb_sa = ((4.0 * np.pi) / (3 * SPLC**2)) * np.power(np.pi*fobs_gw, -4.0/3.0) * np.sum(integrand * np.diff(mbin_edges))
gwb_sa = np.sqrt(gwb_sa)

fig, ax = plot.figax()
xx = fobs_gw * YR
ax.plot(xx, gwb_sa)
plt.show()

# Monte Carlo (MC) Calculation

The GWB can also be calculated explicitly from the full population of binaries in the universe [Sesana et al. 2008, Eq.~10], 
$$h_c^2(f) = \int_0^\infty \!\! dM \, dq \, dz \; \frac{d^4 N}{dM \, dq \, dz \, d\ln f_r} \; h^2(f_r),$$

where the spectral GW strain (*not* characteristic strain) for a circular binary is,

$$h(f_r) = \frac{8}{10^{1/2}} \frac{(G\mathcal{M})^{5/3}}{c^4 d_c} (2\pi f_r)^{2/3}.$$

From [Sesana et al. 2008, Eq.6] we can write,

$$\frac{d^4 N}{dM \, dq \, dz \, d\ln f_r} = \frac{d^3 n_c}{dM \, dq \, dz} \frac{dz}{dt} \frac{dt}{d\ln f_r} \frac{d V_c}{dz}.$$

The standard cosmographic relations are [Hogg 1999],

$$\frac{dz}{dt} = H_0 (1+z) E(z) \\
    \frac{d V_c}{dz} = 4\pi \frac{c}{H_0} \frac{d_c^2}{E(z)} \\
    d_L = d_c \, (1+z)$$

Combining these, we obtain:

$$h_c^2(f) = \int_0^\infty \!\! dM \, dq \, dz \; \frac{d^3 n_c}{dM \, dq \, dz} \, h^2(f_r) \, 4\pi c \, d_c^2 (1+z) \, \frac{f_r}{df_r / dt}.$$

The hardening timescale for a circular, GW-driven binary is:

$$\tau_{GW} \equiv \frac{f_r}{\left[df_r/dt\right]_{GW}} = \frac{5}{96} \frac{c^5}{(G \mathcal{M})^{5/3}} (2\pi f_r)^{-8/3}.$$

Plugging this in to the previous relation gives:

$$h_c^2(f) = \frac{20\pi c^6}{96} \int_0^\infty \!\! dM \, dq \, dz \; \frac{d^3 n_c}{dM \, dq \, dz} \, h^2(f_r) \, \frac{d_c^2 (1+z)}{(G \mathcal{M})^{5/3}} (2\pi f_r)^{-8/3}.$$

<!-- When taking the strain to be due to a circular binary:

$$ h_c^2(f) = \int_0^\infty \!\! dz \; \frac{256 \pi}{10 c^7} \frac{dn_c}{dz} \, \frac{f_r}{df_r / dt} \, (1+z) \, (G \mathcal{M})^{10/3} \, (2\pi f_r)^{4/3}.$$

And for GW-driven evolution,

$$h_c^2(f) = \int_0^\infty \!\! dz \; \frac{4 \pi}{3 c^2} \frac{dn_c}{dz}  \, \frac{(G \mathcal{M})^{5/3}}{(1+z)^{1/3}} \, (2\pi f_r)^{-4/3}.$$ -->

In [ ]:
def gwb_number_from_ndens(ndens, medges, mc_cents, dcom, fro):
    # `fro` = frst_orb
    integrand = ((20*np.pi*(SPLC**6))/96) * ndens * np.diff(medges)
    integrand *= (dcom**2) * (1.0 + redz) * np.power(NWTG * mc_cents, -5.0/3.0)
    integrand = integrand[:, np.newaxis] * np.power(2.0*np.pi*fro, -8.0/3.0)
    return integrand

frst_orb = fobs_gw[np.newaxis, :] * (1.0 + redz) / 2.0
dcom = cosmo.comoving_distance(redz).cgs.value

hs = (8.0 / np.sqrt(10)) * np.power(NWTG * mchirp_cents, 5.0/3.0) / (dcom * (SPLC**4))
hs = hs[:, np.newaxis] * np.power(2*np.pi*frst_orb, 2.0/3.0) 

integrand = gwb_number_from_ndens(ndens, mbin_edges, mchirp_cents, dcom, frst_orb)

sepa_isco = 6 * NWTG * mbin_cents / SPLC**2
frst_orb_isco = utils.kepler_freq_from_sepa(mbin_cents, sepa_isco)
bads = frst_orb > frst_orb_isco[:, np.newaxis]
merged = np.ones_like(integrand)
merged[bads] = 0.0

integrand = np.random.poisson(integrand[..., np.newaxis], integrand.shape + (20,)) * merged[..., np.newaxis]
gwb_mc = np.sum(integrand * (hs**2)[..., np.newaxis], axis=0)

gwb_mc = np.sqrt(gwb_mc)

In [ ]:
fig, ax = plot.figax()
xx = fobs_gw * YR
# ax.plot(xx, gwb_mc[:, np.random.choice(20, 5, replace=False)], alpha=0.2)
ax.plot(xx, np.median(gwb_mc, axis=-1), lw=0.5)
ax.fill_between(xx, *np.percentile(gwb_mc, [25, 75], axis=-1), alpha=0.5)

ax.plot(xx, gwb_sa, 'k--')
plt.show()

# Finite Population Calculation

The number density was calculated from a finite number of binaries, in a finite volume.  Instead of going through the number-density as an intermediate quantity (i.e. binning sample binaries), just use the finite number of binaries directly to calculate the GWB.

$$
    \frac{d^3 n_c}{dM \, dq \, dz} \, dM \, dq \, dz
        \rightarrow \frac{1}{V_c} \sum_i  \delta(M < M_i < M + \Delta M) \cdot \delta(q < q_i < q + \Delta q) \cdot \delta(z < z_i < z + \Delta z) \, F(M, q, z) \\
        \rightarrow \frac{1}{V_c} \sum_i F(M_i \,,\, q_i \,,\, z_i)
$$

In [ ]:
dcom = cosmo.comoving_distance(redz).cgs.value
frst_orb = fobs_gw[np.newaxis, :] * (1.0 + redz) / 2.0
mchirp = utils.chirp_mass_mtmr(masses, mrat)

hs = (8.0 / np.sqrt(10)) * np.power(NWTG * mchirp, 5.0/3.0) / (dcom * (SPLC**4))
hs = hs[:, np.newaxis] * np.power(2*np.pi*frst_orb, 2.0/3.0) 

integrand = ((20*np.pi*(SPLC**6))/96) / vcom
integrand *= (dcom**2) * (1.0 + redz) * np.power(NWTG * mchirp, -5.0/3.0)
integrand = integrand[:, np.newaxis] * np.power(2.0*np.pi*frst_orb, -8.0/3.0)

sepa_isco = 6 * NWTG * masses / SPLC**2
frst_orb_isco = utils.kepler_freq_from_sepa(masses, sepa_isco)
bads = frst_orb > frst_orb_isco[:, np.newaxis]
merged = np.ones_like(integrand)
merged[bads] = 0.0

# integrand = np.random.poisson(integrand, integrand.shape) * merged
integrand = integrand * merged
gwb_fin = np.sum(integrand * (hs**2), axis=0)

gwb_fin = np.sqrt(gwb_fin)

In [ ]:
fig, ax = plot.figax()
xx = fobs_gw * YR
ax.plot(xx, gwb_fin)
# ax.plot(xx, np.median(gwb_mc, axis=-1), lw=0.5)
# ax.fill_between(xx, *np.percentile(gwb_mc, [25, 75], axis=-1), alpha=0.5)

ax.plot(xx, gwb_sa, 'k--')
plt.show()

# Resampling Binned Population

In [ ]:
def extrap_cents_to_edges(grid):
    ndim = grid.ndim
    vals = np.copy(grid)

    # Extrapolate to one more point each left and right, along each axis progressively
    # (A,B,C,...) ==> (A+2,B,C,...) ==> (A+2,B+2,C,...) ==>  ...  ==> (A+2, B+2, C+2, ...)
    for ax in range(ndim):
        vals = np.moveaxis(vals, ax, 0)
        ll = 2*vals[0] - vals[1]
        rr = 2*vals[-1] - vals[-2]
        vals = np.concatenate([[ll], vals, [rr]], axis=0)
        vals = np.moveaxis(vals, 0, ax)

    # Interpolate to mid-points along each axis
    # (A+2,B+2,C+2,...) ==> (A+1,B+2,C+2,...) ==> (A+1,B+1,C,...) ==>  ...  ==> (A+1, B+1, C+1, ...)
    for ax in range(ndim):
        vals = np.moveaxis(vals, ax, 0)
        vals = 0.5 * (vals[:-1] + vals[1:])
        vals = np.moveaxis(vals, 0, ax)

    return vals

In [ ]:
NREALS = 30

_df = frst_orb[0, 0]
frst_orb_edges = np.concatenate([frst_orb[0] - _df/2, [frst_orb[0][-1] + _df/2]])

number = gwb_number_from_ndens(ndens, mbin_edges, mchirp_cents, dcom, frst_orb)
print(f"{number.sum()=:.4e}")

sample_ndens = number / np.diff(mbin_edges)[:, np.newaxis]
sample_ndens = extrap_cents_to_edges(sample_ndens)

## Sample Full Grid

In [ ]:
gwb_resamp_full = np.zeros((fobs_gw.size, NREALS))

for ii in utils.tqdm(range(NREALS)):
    sample_edges = [np.log10(mbin_edges), np.log(frst_orb_edges)]
    vals = kale.sample_grid(sample_edges, sample_ndens, mass=number)
    mm = 10.0 ** vals[0]
    frorb = np.e ** vals[1]

    dcom = cosmo.comoving_distance(redz).cgs.value
    mchirp = utils.chirp_mass_mtmr(mm, mrat)

    hs = (8.0 / np.sqrt(10)) * np.power(NWTG * mchirp, 5.0/3.0) / (dcom * (SPLC**4))
    hs = hs * np.power(2*np.pi*frorb, 2.0/3.0) 

    sepa_isco = 6 * NWTG * mm / SPLC**2
    frst_orb_isco = utils.kepler_freq_from_sepa(mm, sepa_isco)
    bads = frorb > frst_orb_isco
    merged = np.ones_like(bads, dtype=float)
    merged[bads] = 0.0

    fogw = frorb * 2.0 / (1.0 + redz)
    hs = merged * (hs**2)
    gwb_resamp_full[:, ii], *_ = sp.stats.binned_statistic(fogw, hs, statistic='sum', bins=fobs_gw_edges)
    gwb_resamp_full[:, ii] = np.sqrt(gwb_resamp_full[:, ii])

In [ ]:
fig, ax = plot.figax()
xx = fobs_gw * YR
ax.plot(xx, gwb_sa, 'k--', lw=1.5, alpha=0.5)
ax.plot(xx, gwb_fin, 'k:', lw=2.0, alpha=0.5)
gwb_mc_med = np.median(gwb_mc, axis=-1)
ax.plot(xx, gwb_mc_med, color='b', lw=2.0, alpha=0.5)

med = np.median(gwb_resamp_full, axis=-1)
ax.plot(xx, med, lw=0.5, color='r')
ax.fill_between(xx, *np.percentile(gwb_resamp_full, [25, 75], axis=-1), alpha=0.5, color='r')

tw = ax.twinx()
tw.plot(xx, gwb_sa/med, 'r--', alpha=0.5)
tw.plot(xx, gwb_mc_med/med, 'b--', alpha=0.5)
plt.show()

## Sample 'Outliers'

In [ ]:
gwb_resamp_out = np.zeros((fobs_gw.size, NREALS))
sample_threshold = 1e3

for ii in utils.tqdm(range(NREALS)):
    sample_edges = [np.log10(mbin_edges), np.log(frst_orb_edges)]
    vals, weights = kale.sample_outliers(sample_edges, sample_ndens, sample_threshold, mass=number) 
    
    mm = 10.0 ** vals[0]
    frorb = np.e ** vals[1]

    dcom = cosmo.comoving_distance(redz).cgs.value
    mchirp = utils.chirp_mass_mtmr(mm, mrat)

    hs = (8.0 / np.sqrt(10)) * np.power(NWTG * mchirp, 5.0/3.0) / (dcom * (SPLC**4))
    hs = hs * np.power(2*np.pi*frorb, 2.0/3.0) 

    sepa_isco = 6 * NWTG * mm / SPLC**2
    frst_orb_isco = utils.kepler_freq_from_sepa(mm, sepa_isco)
    bads = frorb > frst_orb_isco
    merged = np.ones_like(bads, dtype=float)
    merged[bads] = 0.0

    fogw = frorb * 2.0 / (1.0 + redz)
    hs = merged * (hs**2) * weights
    gwb_resamp_out[:, ii], *_ = sp.stats.binned_statistic(fogw, hs, statistic='sum', bins=fobs_gw_edges)
    gwb_resamp_out[:, ii] = np.sqrt(gwb_resamp_out[:, ii])


In [ ]:
fig, ax = plot.figax()
xx = fobs_gw * YR
ax.plot(xx, gwb_sa, 'k--', lw=1.5, alpha=0.5)
ax.plot(xx, gwb_fin, 'k:', lw=2.0, alpha=0.5)
gwb_mc_med = np.median(gwb_mc, axis=-1)
ax.plot(xx, gwb_mc_med, color='b', lw=1.0, alpha=0.5)

# ax.plot(xx, gwb_resamp_out[:, 0], 'r-')
med = np.median(gwb_resamp_out, axis=-1)
ax.plot(xx, med, lw=0.5, color='r')
ax.fill_between(xx, *np.percentile(gwb_resamp_out, [25, 75], axis=-1), alpha=0.5, color='r')

med = np.median(gwb_resamp_full, axis=-1)
ax.plot(xx, med, lw=0.5, color='r', alpha=0.5)
ax.fill_between(xx, *np.percentile(gwb_resamp_full, [25, 75], axis=-1), alpha=0.25, color='r')

# tw = ax.twinx()
# tw.plot(xx, gwb_sa/med, 'r--', alpha=0.5)
# tw.plot(xx, gwb_mc_med/med, 'b--', alpha=0.5)
plt.show()